*Laetitia BERNE* <br>*Gilles DECKNACHE* <br>*Romain JALBERT*

# Homework n°1

### Importation des modules nécessaires


In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

import os, sys
sys.path.insert(0,os.path.abspath(os.path.join(os.path.abspath(''),'../utilities')))
import optmodel_utilities as opt_utils   

import time

### Création du modèle avec la relaxation linéaire

In [2]:
def Create_model():

    model = AbstractModel()
    
    # Paramètre : nombre de sommets dans le graphe
    model.n = Param()
    
    # Ensemble des sommets du graphe
    model.V = RangeSet(model.n)
    
    # Paramètre : le coût d'installation d'un relai dans une localisation donnée
    model.c = Param(model.V)
    
    # Ensemble des arcs présents dans le graphe
    model.E = Set(within = model.V*model.V)
    
    
    # Variables
    model.x = Var(model.V, within = NonNegativeReals) #NonNegativeReals
    
    
    # Fonction Objectif : minimisation de la somme des coûts
    def Objectif(model):
        return sum(model.c[i]*model.x[i] for i in model.V)
    
    model.cout_total = Objective(rule=Objectif, sense=minimize)
    
    
    # Contraintes
    def regle_Contrainte(model, i, j):
        return model.x[i] + model.x[j] >= 1
    
    model.contrainte = Constraint(model.E, rule=regle_Contrainte)
   
    
    return model

### Création d'une instance et résolution 

In [3]:
# Choix du solver
optsolver = opt_utils.create_solver()

# Création du modèle
model = Create_model()

# Chargement des données depuis un fichier (et création d'une instance)
instance = model.create_instance('MRP_40.dat')        # MRP_20 ; MRP_40 ; MRP_80 ; MRP_160

#This command allow to print the full structure of the instance/model
#instance.pprint()

#solving the problem
results = optsolver.solve(instance)


### Affichage des résultats de la relaxation linéaire du problème

In [4]:
if (results.solver.status == SolverStatus.ok) and \
    (results.solver.termination_condition == TerminationCondition.optimal):

    opt_utils.print_objective_value(instance)
    opt_utils.print_point_from_model(instance)
    
    # dictionnaire pour stocker les valeurs des variables
    xi_values = {}

    for var in instance.component_objects(Var, active=True):
        varobject = getattr(instance, str(var))
        for index in varobject:
            xi_values[index] = varobject[index].value
    
    n = instance.n.value
    ensemble_E = instance.E.data()
    
else :
    print("Some problem occurred. Solver terminated with condition ", results.solver.termination_condition)

OBJ:  cout_total  =  104.0
x [ 1 ] =  0.5
x [ 2 ] =  0.5
x [ 3 ] =  0.5
x [ 4 ] =  0.5
x [ 5 ] =  0.5
x [ 6 ] =  0.5
x [ 7 ] =  0.5
x [ 8 ] =  0.5
x [ 9 ] =  0.5
x [ 10 ] =  0.5
x [ 11 ] =  0.5
x [ 12 ] =  0.5
x [ 13 ] =  0.5
x [ 14 ] =  0.5
x [ 15 ] =  0.5
x [ 16 ] =  0.5
x [ 17 ] =  0.5
x [ 18 ] =  0.5
x [ 19 ] =  0.5
x [ 20 ] =  0.5
x [ 21 ] =  0.5
x [ 22 ] =  0.5
x [ 23 ] =  0.5
x [ 24 ] =  0.5
x [ 25 ] =  0.5
x [ 26 ] =  0.5
x [ 27 ] =  0.5
x [ 28 ] =  0.5
x [ 29 ] =  0.5
x [ 30 ] =  0.5
x [ 31 ] =  0.5
x [ 32 ] =  0.5
x [ 33 ] =  0.5
x [ 34 ] =  0.5
x [ 35 ] =  0.5
x [ 36 ] =  0.5
x [ 37 ] =  0.5
x [ 38 ] =  0.5
x [ 39 ] =  0.5
x [ 40 ] =  0.5


### Randomized rounding

In [5]:
start_time = time.process_time()


# Création de l'ensemble C qui contiendra les sommets où on va installer une antenne
C = []

# Conversion de l'ensemble des arcs en une liste, que l'on va pouvoir modifier (c'était un tuple jusqu'à présent)
E = list(ensemble_E)    

In [6]:
# E_new est un dictionnaire : 
# clé : arcs de E 
# valeur : 'oui' si l'arc est couvert
#          'Non' sinon



E_new={}

# Initialisation de E_new
for arc in E:
    E_new[arc]='non'


nb_arc = len(E) # c'est le nombre d'arc qu'il reste à couvrir


# On ajoute à C les sommets qui ont déjà la valeur 1 et on actualise E et nb_arc
for i in range(1, n+1):
    
    if xi_values[i] == 1 :
        C.append(i)
                
        for indice_arc in range(len(E)) :
            if E[indice_arc][0] == i or E[indice_arc][1] == i : # Si indice arc est lié au sommet i 
                if E_new[E[indice_arc]]=='non': # et que l'arc n'était pas couvert
                    E_new[E[indice_arc]]='oui' # alors on couvre l'arc
                    nb_arc = nb_arc - 1 # et on a un arc en moins à couvrir

In [7]:
from random import random,choice

# On récupère la liste des sommets



V = list(instance.V.data())

# On génère un nombre entre 0 et 1
r = random()

for noeud in C:
    V.remove(noeud)

# On veut couvrir tout le graphe 

while nb_arc != 0:
    
    # On génère un nombre entre 0 et 1
    r = random()
    
    # On choisi un noeud au hasard
    noeud = choice(V)
    
    # On arrondit le neoud avec la proba xi 
    if xi_values[noeud] > r :
        
        C.append(noeud)
        V.remove(noeud)
        
        # Actualisation de E_new et du nombre d'arc
        for indice_arc in range(len(E)) :
            if E[indice_arc][0] == noeud or E[indice_arc][1] == noeud :
                if E_new[E[indice_arc]]=='non':
                    E_new[E[indice_arc]]='oui'
                    nb_arc -= 1 
                    
total_time = time.process_time() - start_time
                    


# recuperation des valeurs des couts
def extract_values(m, i):
    x_ix = value(m.c[i])
    return x_ix

c = [extract_values(instance, i) for i in range(1,n+1)] 
S=0
for i in C: 
    S+= c[i-1]
    
print("Solution Randomized Rounding : ",C)

print("Objective function : ",S)

print("time :", total_time)


Solution Randomized Rounding :  [33, 29, 1, 18, 9, 15, 36, 8, 37, 26, 16, 20, 30, 24, 12, 28, 6, 39, 27, 21, 2, 4, 3, 25, 40, 19, 35, 13, 7, 11, 31, 10, 14, 34, 32, 38, 23]
Objective function :  192
time : 0.03769700000000009


### Very Large Scale Neighborhood (VLSN)

In [8]:
# Définition de la taille du voisinage
k = 2
# 2 est un petit voisinage, mais qui est suffisant à la méthode pour converger en très peu d'itérations

In [9]:
# Création du modèle avec la contrainte supplémentaire
def Create_model_VLSN(k, C):

    model = AbstractModel()
    
    # Paramètre : nombre de sommets dans le graphe
    model.n = Param()
    
    # Ensemble des sommets du graphe
    model.V = RangeSet(model.n)
    
    # Paramètre : le coût d'installation d'un relai dans une localisation donnée
    model.c = Param(model.V)
    
    # Ensemble des arcs présents dans le graphe
    model.E = Set(within = model.V*model.V)
    
    # Création de l'ensemble S, qui contient les sommets qui valent 1 dans la solution de référence trouvée avec
    # le ramdomized rounding
    model.S = Set(initialize=C)
    
    
    # Variables
    model.x = Var(model.V, within = Binary) 
    
    
    # Fonction Objectif : minimisation de la somme des coûts
    def Objectif(model):
        return sum(model.c[i]*model.x[i] for i in model.V)
    
    model.cout_total = Objective(rule=Objectif, sense=minimize)
    
    
    # Contraintes
    def regle_Contrainte(model, i, j):
        return model.x[i] + model.x[j] >= 1
    
    model.contrainte = Constraint(model.E, rule=regle_Contrainte)
    
    def regle_supplementaire(model) :
        return sum(model.x[i] for i in model.V-model.S) + sum(1 - model.x[i] for i in model.S) <= k
    model.contrainte_supplementaire = Constraint(rule=regle_supplementaire)
   
    
    return model

In [10]:
# Choix du solver

start = time.process_time()

optsolver = opt_utils.create_solver()

# Création du modèle et itération 
for iteration in range(5) :
    model_VLSN = Create_model_VLSN(k, C)

    # Chargement des données depuis un fichier (et création d'une instance)
    instance_VLSN = model_VLSN.create_instance('MRP_40.dat')        # MRP_20 ; MRP_40 ; MRP_80 ; MRP_160

    #solving the problem
    results_VLSN = optsolver.solve(instance_VLSN)

    if (results_VLSN.solver.status == SolverStatus.ok) and \
        (results_VLSN.solver.termination_condition == TerminationCondition.optimal):


        # dictionnaire pour stocker les valeurs des variables
        x = {}

        for var in instance_VLSN.component_objects(Var, active=True):
            varobject = getattr(instance_VLSN, str(var))
            for index in varobject:
                x[index] = varobject[index].value

        C = [cle for cle in x if x[cle]==1]

    else :
        print("Some problem occurred. Solver terminated with condition ", results_VLSN.solver.termination_condition)
        

total_time = time.process_time() - start_time       
print("Local search time ", total_time)
opt_utils.print_objective_value(instance_VLSN)
opt_utils.print_point_from_model(instance_VLSN)


Local search time  0.38135200000000014
OBJ:  cout_total  =  168.0
x [ 1 ] =  1.0
x [ 2 ] =  1.0
x [ 3 ] =  1.0
x [ 4 ] =  1.0
x [ 5 ] =  1.0
x [ 6 ] =  1.0
x [ 7 ] =  1.0
x [ 8 ] =  0.0
x [ 9 ] =  1.0
x [ 10 ] =  1.0
x [ 11 ] =  1.0
x [ 12 ] =  1.0
x [ 13 ] =  1.0
x [ 14 ] =  1.0
x [ 15 ] =  1.0
x [ 16 ] =  1.0
x [ 17 ] =  0.0
x [ 18 ] =  1.0
x [ 19 ] =  1.0
x [ 20 ] =  1.0
x [ 21 ] =  1.0
x [ 22 ] =  1.0
x [ 23 ] =  1.0
x [ 24 ] =  1.0
x [ 25 ] =  1.0
x [ 26 ] =  1.0
x [ 27 ] =  1.0
x [ 28 ] =  0.0
x [ 29 ] =  1.0
x [ 30 ] =  0.0
x [ 31 ] =  1.0
x [ 32 ] =  1.0
x [ 33 ] =  1.0
x [ 34 ] =  1.0
x [ 35 ] =  0.0
x [ 36 ] =  1.0
x [ 37 ] =  1.0
x [ 38 ] =  1.0
x [ 39 ] =  1.0
x [ 40 ] =  1.0


### Comparaison

*Pour le problème à 10 sommets :* <br> L'optimum vaut 17, avec C = [1, 2, 5, 6, 9]. <br>
La relaxation continue nous donne une borne inférieure qui vaut 15. Le coût avec randomized rounding est 24 et celui du VLSN est de 17. On arrive donc bien à trouver l'optimum, en très peu d'itérations.


*Pour le problème à 20 sommets :* <br> La relaxation continue nous donne une borne inférieure qui vaut 52. Le coût avec randomized rounding est 92 et celui du VLSN est de 70. On arrive donc bien à trouver une solution performante en très peu d'itérations. Ce problème a donc pour bornes [52, 70].

*Pour le problème à 40 sommets :* <br> La relaxation continue nous donne une borne inférieure qui vaut 104. Le coût avec randomized rounding est 199 et celui du VLSN est de 171. On arrive donc bien à trouver une solution performante en très peu d'itérations. Ce problème a donc pour bornes [104, 171].

*Pour les problèmes à 80 et 160 sommets, la version gratuite de Pyomo ne nous permet pas d'obtenir le résultat car ces modèles ont trop de contraintes*


On remarque que randomized rounding est plus court à l'execution que VLSN.